In [47]:
import pandas as pd
from unidecode import unidecode
import re
import ast
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [48]:
import warnings

#Suppress all warnings
warnings.filterwarnings("ignore")


## DF TSHIRTS:

In [49]:
tshirt = pd.read_csv('/Users/admin/Desktop/IRONHACK/8.WEEK/final_project/notebooks/datos/final_categorias/tshirts.csv')
tshirt.reset_index(inplace = True)


In [50]:
tshirt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         729 non-null    int64  
 1   Unnamed: 0    729 non-null    int64  
 2   product_name  720 non-null    object 
 3   price         729 non-null    object 
 4   desciption    729 non-null    object 
 5   description   0 non-null      float64
dtypes: float64(1), int64(2), object(3)
memory usage: 34.3+ KB


In [51]:
tshirt.drop(columns=['Unnamed: 0','description'], inplace=True)


In [52]:
tshirt.head()

,index,product_name,price,desciption
0,0,CAMISETA ALGODÓN MANGA SISA,"8,95 EUR",['Trabajamos con programas de seguimiento para...
1,1,CAMISETA ALGODÓN MANGA SISA,"8,95 EUR",['Trabajamos con programas de seguimiento para...
2,2,CAMISETA ALGODÓN MANGA SISA,"8,95 EUR",['Trabajamos con programas de seguimiento para...
3,3,CAMISETA ALGODÓN BÁSICA,"6,95 EUR",['Trabajamos con programas de seguimiento para...
4,4,CAMISETA ALGODÓN,"12,95 EUR",['Trabajamos con programas de seguimiento para...


In [53]:
# Remove accents with unidecode
def remove_accents(text):
    if isinstance(text, str):
        return unidecode(text)
    return text

In [54]:
# Aplicar la función a todo el DataFrame
tshirt = tshirt.applymap(remove_accents)

In [55]:
tshirt.head()

,index,product_name,price,desciption
0,0,CAMISETA ALGODON MANGA SISA,"8,95 EUR",['Trabajamos con programas de seguimiento para...
1,1,CAMISETA ALGODON MANGA SISA,"8,95 EUR",['Trabajamos con programas de seguimiento para...
2,2,CAMISETA ALGODON MANGA SISA,"8,95 EUR",['Trabajamos con programas de seguimiento para...
3,3,CAMISETA ALGODON BASICA,"6,95 EUR",['Trabajamos con programas de seguimiento para...
4,4,CAMISETA ALGODON,"12,95 EUR",['Trabajamos con programas de seguimiento para...


In [56]:
tshirt.shape

(729, 4)

In [57]:
tshirt.dropna(subset=["product_name"], inplace=True)

In [58]:
tshirt.shape

(720, 4)

### Clean and create new columns

- Create column made_in with the origin of the clothes.
- Create column composition of clothes.
- Create column recycled or not and wich material is recycled. 
- Create column for OCS & RCS. 
- Clean price, estandar, product_name & remove spaces, accents, symbols, quotes, etc. 

#### Column Product_name: 

In [59]:
def convertir_formato(texto):
    return texto.lower().title()

In [60]:
tshirt["product_name"] = tshirt["product_name"].apply(convertir_formato)

In [61]:
tshirt.head()

,index,product_name,price,desciption
0,0,Camiseta Algodon Manga Sisa,"8,95 EUR",['Trabajamos con programas de seguimiento para...
1,1,Camiseta Algodon Manga Sisa,"8,95 EUR",['Trabajamos con programas de seguimiento para...
2,2,Camiseta Algodon Manga Sisa,"8,95 EUR",['Trabajamos con programas de seguimiento para...
3,3,Camiseta Algodon Basica,"6,95 EUR",['Trabajamos con programas de seguimiento para...
4,4,Camiseta Algodon,"12,95 EUR",['Trabajamos con programas de seguimiento para...


In [62]:
tshirt["product_name"] = tshirt["product_name"].str.strip() #delete null space infront of and behind each row
tshirt["product_name"] = tshirt["product_name"].str.lstrip() #delete null space left hand


In [63]:
# Alinea el texto a la izquierda y establece una anchura fija
tshirt = tshirt.applymap(lambda x: f'{x:<50}' if isinstance(x, str) else x)

##### Create rgex pattern to simplify the analysis

In [64]:
patt_tshirt = { r"[Ee]nzimatico" : "Camiseta Algodon Enzimatico",
                r"[Cc]amiseta Algodon": "Camiseta Algodon Basica",
                r"[Ss]udadera" : "Sudadera",
                r"[Cc]amiseta [Hh]eavy" : "Camiseta Heavy Cotton",
                r"[Rr]ib" : "Camiseta Rib", 
                r"[Oo]versize" : "Camiseta Oversize",
                r"[Pp]oliamida" : "Camiseta Poliamida", 
                r"[Tt]op" : "Top Crop"
              }

In [65]:
def categorize_activities(text, patt_tshirt):
    if isinstance(text, str):  # Check if the value is a text string
        for pattern, category in patt_tshirt.items():
            if re.search(pattern, text):
                return category
    return "Camiseta Temporada"  # If it does not match any category, "Other" is returned.

In [66]:
# Clear invalid values in the "activity" column
tshirt["product_name"] = tshirt["product_name"].fillna("empty")  # Fill NaN values with an empty string
tshirt["product_name"] = tshirt["product_name"].astype(str)  # Convert all values to text strings
# Apply the function to categorize activities
tshirt["product_name"] = tshirt["product_name"].apply(lambda x: categorize_activities(x, patt_tshirt))
tshirt["product_name"].unique()

array(['Camiseta Algodon Basica', 'Camiseta Temporada',
       'Camiseta Poliamida', 'Camiseta Heavy Cotton', 'Camiseta Rib',
       'Camiseta Algodon Enzimatico', 'Top Crop', 'Sudadera',
       'Camiseta Oversize'], dtype=object)

In [67]:
# Aplicar la función a la columna 'product_name'
tshirt['product_name'].value_counts()

product_name
Camiseta Temporada             338
Camiseta Algodon Basica        100
Camiseta Rib                    93
Sudadera                        67
Camiseta Poliamida              44
Camiseta Oversize               27
Camiseta Heavy Cotton           18
Top Crop                        18
Camiseta Algodon Enzimatico     15
Name: count, dtype: int64

#### Column made in:

In [68]:
#Function for extract made in
def extract_origin(desciption):
    match = re.search(r'hecho\s+en\s+(\w+)', desciption, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return None

In [69]:
# Apply the function to the column description and create a new column called made_in
tshirt['made_in'] = tshirt['desciption'].apply(extract_origin)

In [70]:
tshirt['made_in'].value_counts()

made_in
Bangladesh    291
Marruecos     192
Turquia       123
Portugal       68
Pakistan       30
China          10
India           2
Camboya         2
Egipto          2
Name: count, dtype: int64

#### Column estandar:

In [71]:
# Regex pattern
patron_regex = re.compile(r'(?:estandares\s(.*))', re.IGNORECASE)

In [72]:
#Function to extract standar
def extract_estandar(desciption):
    # Desempaquetar la cadena de texto de la lista
    description = ast.literal_eval(desciption) #convertir un string a lista
    match = patron_regex.search(" ".join(description[0:1]))
    #print(" ".join(description[0:1]))
    #print(match)
    if match:
        return match.group(1)
    else:
        return None

In [73]:
# Apply the function to the column description and create a new column called made_in
tshirt['estandar'] = tshirt['desciption'].apply(extract_estandar)

In [74]:
tshirt['estandar'].value_counts()

estandar
sociales, medioambientales y de seguridad y salud de nuestros productos. Para evaluar su cumplimiento hemos desarrollado un programa de auditorias y planes de mejora continua.    720
Name: count, dtype: int64

In [75]:
def check_create(text):
    palabras_clave = ['estandar', 'sociales', 'medioambientales', 'seguridad y salud']
    for palabra in palabras_clave:
        if palabra in text:
            return True
    return False

In [76]:
tshirt['standard_soc_ma'] = tshirt['estandar'].apply(check_create)

In [77]:
tshirt['standard_soc_ma'].value_counts()

standard_soc_ma
True    720
Name: count, dtype: int64

In [78]:
tshirt.drop(columns=['estandar'], inplace=True)

#### Column composition:

In [79]:
pattern = re.compile(r"'[%]['']'")

In [80]:
def extract_composition_materiales_4(df):
    
    df["comprobacion"] = np.nan
    
    for idn in df.index:
        
        desciption = df.loc[idn, "desciption"]
    
        if desciption != "[]":
            # Desempaquetar la cadena de texto de la lista
            res = desciption.strip('][').split(', ')
            s1 = res[2:4][0] #Posicion donde esta ubicado en la lista
            #print(s1)
            patron = r'(\d+)%\s*([a-zA-Z]+)'
            # Buscamos todas las coincidencias en la cadena
            coincidencias = re.findall(patron, s1)
            #print(coincidencias)
        
            df["comprobacion"][idn] = coincidencias
            
            for porcentaje, material in coincidencias:
                if material in df.columns:
                    df[material][idn] = porcentaje
                else:
                    df[material]= np.nan
                    df[material][idn] = porcentaje
        else:
            continue
                
    return df

In [81]:
apply_function = extract_composition_materiales_4(tshirt)

In [82]:
#tshirt.info()

In [83]:
#tshirt.head()

In [84]:
# Crear una nueva columna para contar el número de materiales en cada lista
tshirt['num_materials'] = tshirt['comprobacion'].apply(lambda x: len(x) if isinstance(x, list) else 0)

In [85]:
# Crear una nueva columna para asignar el main material de cada producto.

tshirt['material'] = tshirt['comprobacion'].apply(
    lambda x: x[0][1] if isinstance(x, list) and len(x) > 0 and len(x[0]) > 1 else None
)

In [86]:
tshirt["material"].value_counts()

material
algodon      603
poliamida     46
poliester     32
viscosa       25
liocel         6
modal          4
acrilico       2
seda           2
Name: count, dtype: int64

In [87]:
# Crear una nueva columna para asignar el material secundario de cada producto.
tshirt['num_material'] = tshirt['comprobacion'].apply(lambda x: int(x[0][0]) if len(x) > 0 else None)

In [88]:
# Crear una nueva columna para contar el % de material para ese producto
tshirt['material_secundario'] = tshirt['comprobacion'].apply(lambda x: x[1][1] if len(x) > 1 else None)

In [89]:
# Crear una nueva columna para contar el % de material para ese producto
tshirt['num_material_secundario'] = tshirt['comprobacion'].apply(lambda x: int(x[1][0]) if len(x) > 1 else None)

In [90]:
columns_to_keep = ["comprobacion", "material", "num_materials", "num_material", "material_secundario", "num_material_secundario"]

filtered_df = tshirt[columns_to_keep]
#filtered_df.tail(20)

In [91]:
tshirt["num_materials"].value_counts()

num_materials
1    392
2    293
3     30
4      5
Name: count, dtype: int64

#### Column recycled:

In [92]:
def recycled_2(df):
    
    # Inicializar columnas adicionales para cada tipo de material
    for material in ["OCS", "RCS", "reciclado"]:
        df[material] = np.nan
    
    # Inicializar columna de comprobación
    df["comprobacion2"] = np.nan
    
    # Iterar sobre cada índice del DataFrame
    for idn in df.index:
        # Obtener la descripción de la fila actual
        desciption = df.loc[idn, "desciption"]
        
        # Lista para almacenar los materiales encontrados
        materiales_encontrados = []
        
        # Verificar si la descripción no está vacía
        if desciption != "[]":
            # Desempaquetar la cadena de texto de la lista
            res = desciption.strip('][').split(', ')
            
            # Recorrer cada elemento de la descripción
            for item in res:
                # Buscar coincidencias con el patrón en la descripción
                matches = re.findall(r"(\d+% .*?(?:OCS|RCS|reciclado|cultivo orgánico))", item)
                
                # Iterar sobre las coincidencias encontradas
                for match in matches:
                    # Extraer el porcentaje y el tipo de material
                    porcentaje, material = match.split(None, 1)
                    
                    # Limpiar el tipo de material
                    material = material.strip()
                    
                    # Asignar el porcentaje al tipo de material en la fila actual
                    df.at[idn, material] = porcentaje
                    
                    # Agregar el material a la lista de materiales encontrados
                    materiales_encontrados.append(material)
                    
        # Asignar la lista de materiales encontrados a la columna de comprobación
        df.at[idn, "comprobacion2"] = materiales_encontrados
                
    return df

In [93]:
aply_recicled = recycled_2(tshirt)

In [94]:
#tshirt.info()

In [95]:
columns_to_keep = ["comprobacion2", "algodon de cultivo organico certificado OCS","poliester reciclado","algodon reciclado","poliamida reciclada certificada RCS", "acrilico reciclado", "elastano reciclado"]

filtered_df = tshirt[columns_to_keep]
#filtered_df.tail(20)

In [96]:
#funcion para identificar la palabra en el df y asignar la certificacion
def obtener_certificacion(lista):
    if 'algodon de cultivo organico certificado OCS' in lista \
        or 'algodon de cultivo organico' in lista \
        or 'algodon organico' in lista \
        or 'algodon certificado OCS' in lista \
        or 'algodon OCS' in lista \
        or 'algodon reciclado certificado' in lista \
        or 'algodon reciclado' in lista:
        return 'OCS'
    elif 'poliamida reciclada certificada RCS' in lista \
        or 'reciclada' in lista:
        return 'RCS'
    else:
        return "None"

In [97]:
tshirt['certificacion'] = tshirt['comprobacion2'].apply(obtener_certificacion)

In [98]:
#tshirt.info()

In [99]:
columns_to_keep = ["comprobacion2", "algodon de cultivo organico certificado OCS","poliester reciclado","algodon reciclado","poliamida reciclada certificada RCS", "acrilico reciclado", "elastano reciclado", "certificacion"]

filtered_df = tshirt[columns_to_keep]
#filtered_df.tail(20)

In [100]:
tshirt["certificacion"].value_counts()

certificacion
None    526
OCS     159
RCS      35
Name: count, dtype: int64

#### Column Price: 

In [101]:
def remove_currency(df):
    df["price"] = df["price"].replace("EUR", "", regex=True)
    return df

In [102]:
remove = remove_currency(tshirt)

#### Column certificate por Intertek 193341:

In [103]:
def add_intertek_column(df):
    pattern = re.compile(r'Certificado\s+por\s+Intertek\s+193341', re.IGNORECASE)
    df['intertek'] = df['desciption'].apply(lambda x: bool(pattern.search(x)))
    
    return df

In [104]:
apply3 = add_intertek_column(tshirt)

In [105]:
tshirt["intertek"].value_counts()

intertek
False    494
True     226
Name: count, dtype: int64

#### Etiquetar OCS & RCS

In [106]:
def etiquetar(df):
    df['OCS'] = ''  # Inicializar la columna OCS
    df['RCS'] = ''  # Inicializar la columna RCS

    for index, row in df.iterrows():
        comprobacion = row['comprobacion2']
        if isinstance(comprobacion, str):
            if 'organico' in comprobacion:
                df.at[index, 'OCS'] = 'OCS'
            if 'reciclado' in comprobacion:
                df.at[index, 'RCS'] = 'RCS'
        elif isinstance(comprobacion, list):
            for item in comprobacion:
                if 'organico' in item:
                    df.at[index, 'OCS'] = 'OCS'
                if 'reciclado' in item:
                    df.at[index, 'RCS'] = 'RCS'

    return df


In [107]:
tshirt = etiquetar(tshirt)

In [108]:
tshirt = etiquetar(tshirt)

In [109]:
# Crear una función para obtener el porcentaje correspondiente
def obtener_porcentaje(row):
    if row['certificacion'] == 'OCS':
        return row['algodon de cultivo organico certificado OCS']
    elif row['certificacion'] == 'RCS':
        return row['poliamida reciclada certificada RCS']
    else:
        return None

In [110]:
tshirt['porcentaje'] = tshirt.apply(obtener_porcentaje, axis=1)

In [111]:
# Crear una función para buscar materiales reciclados en la lista
def encontrar_material_reciclado(lista):
    for material in lista:
        if 'algodon reciclado' in material:
            return 'algodon reciclado'
        elif 'poliester reciclado' in material:
            return 'poliester reciclado'
        elif 'poliamida reciclada' in material:
            return 'poliamida reciclada'
    return None

In [112]:
tshirt['reciclado'] = tshirt['comprobacion2'].apply(encontrar_material_reciclado)

In [113]:
tshirt['reciclado'].value_counts()

reciclado
algodon reciclado      50
poliester reciclado    40
poliamida reciclada    35
Name: count, dtype: int64

### EDA

In [114]:
# Obtener el recuento de los diferentes tipos de materiales reciclados
certificacion_counts = tshirt['certificacion'].value_counts(dropna=True)

# Crear la tabla dinámica
pivot_table23 = pd.DataFrame(certificacion_counts, columns=['Recuento'])

# Calcular el número total de productos
total_products = len(tshirt)

# Calcular el número total de productos con certificaciones (excluyendo NaN)
total_certified_products = tshirt['certificacion'].notna().sum()

# Calcular el porcentaje de productos con certificaciones
percentage_certified = (total_certified_products / total_products) * 100

# Agregar el porcentaje a la tabla dinámica
pivot_table23.loc['Cert_T-shirt'] = [f"{percentage_certified:.2f}%"]

# Mostrar la tabla
pivot_table23

,Recuento
Cert_T-shirt,100.00%


In [115]:
#pivot_table23.to_csv('certif_tshirt.csv')

In [116]:
# Obtener el recuento de los diferentes tipos de materiales reciclados
certification = tshirt['certificacion'].value_counts()

# Crear la tabla dinámica
pivot_table2 = pd.DataFrame(certification)
pivot_table2

,count
certificacion,
None,526
OCS,159
RCS,35


In [117]:
pivot_table2.to_csv('total_cert.csv')

In [118]:
# Obtener el recuento de los diferentes tipos de certificaciones por producto
pivot_table_por_producto = tshirt.groupby('product_name')['certificacion'].value_counts().unstack(fill_value=0)

# Esta línea agrega un total por producto, si no deseas este total puedes omitir la línea
pivot_table_por_producto['Total'] = pivot_table_por_producto.sum(axis=1)

# Mostrar la tabla dinámica por producto
pivot_table_por_producto

certificacion,None,OCS,RCS,Total
product_name,,,,
Camiseta Algodon Basica,79,21,0,100
Camiseta Algodon Enzimatico,15,0,0,15
Camiseta Heavy Cotton,10,8,0,18
Camiseta Oversize,18,8,1,27
Camiseta Poliamida,11,0,33,44
Camiseta Rib,78,15,0,93
Camiseta Temporada,263,74,1,338
Sudadera,40,27,0,67
Top Crop,12,6,0,18


In [119]:
# Calcular el total de certificaciones para todos los productos
total_certificaciones = pivot_table_por_producto.sum()

# Agregar el total al final de la tabla dinámica
pivot_table_por_producto.loc['Total'] = tshirt['product_name'].value_counts()
pivot_table_por_producto

certificacion,None,OCS,RCS,Total
product_name,,,,
Camiseta Algodon Basica,79.0,21.0,0.0,100.0
Camiseta Algodon Enzimatico,15.0,0.0,0.0,15.0
Camiseta Heavy Cotton,10.0,8.0,0.0,18.0
Camiseta Oversize,18.0,8.0,1.0,27.0
Camiseta Poliamida,11.0,0.0,33.0,44.0
Camiseta Rib,78.0,15.0,0.0,93.0
Camiseta Temporada,263.0,74.0,1.0,338.0
Sudadera,40.0,27.0,0.0,67.0
Top Crop,12.0,6.0,0.0,18.0


In [120]:
# Calcular el porcentaje de certificaciones en relación al total de todas las filas
total_filas = pivot_table_por_producto.iloc[:-1, :].sum().sum()
porcentaje_total = pivot_table_por_producto.div(total_filas) * 100

# Calcular el total de todas las certificaciones
total_certificaciones = pivot_table_por_producto.sum().sum()

# Calcular el porcentaje total de cada tipo de certificación en relación al total de todas las certificaciones
porcentaje_total = pivot_table_por_producto / total_certificaciones * 100

porcentaje_total = porcentaje_total.round(2)

# Mostrar la tabla de porcentajes formateada
porcentaje_total
porcentaje_total

certificacion,None,OCS,RCS,Total
product_name,,,,
Camiseta Algodon Basica,5.49,1.46,0.00,6.94
Camiseta Algodon Enzimatico,1.04,0.00,0.00,1.04
Camiseta Heavy Cotton,0.69,0.56,0.00,1.25
Camiseta Oversize,1.25,0.56,0.07,1.88
Camiseta Poliamida,0.76,0.00,2.29,3.06
Camiseta Rib,5.42,1.04,0.00,6.46
Camiseta Temporada,18.26,5.14,0.07,23.47
Sudadera,2.78,1.88,0.00,4.65
Top Crop,0.83,0.42,0.00,1.25


In [121]:
porcentaje_total.to_csv('porcentaje_total_camiseta.csv')

In [122]:
# Obtener el recuento de los diferentes tipos de materiales reciclados
reciclado_counts = tshirt['reciclado'].value_counts()

# Crear la tabla dinámica
pivot_table = pd.DataFrame(reciclado_counts)
pivot_table

,count
reciclado,
algodon reciclado,50
poliester reciclado,40
poliamida reciclada,35


In [123]:
#pivot_table.to_csv('pivot_table_reciclado_camisetas')

In [124]:
# Obtener el recuento de los diferentes tipos de materiales reciclados
reciclado_counts = tshirt['reciclado'].value_counts()

# Crear la tabla dinámica a partir de reciclado_counts
pivot_table = pd.DataFrame(reciclado_counts).reset_index()
pivot_table.columns = ['reciclado', 'count']

# Calcular el recuento total de productos
total_productos = tshirt.shape[0]

# Agregar una columna para el porcentaje respecto al total de productos
pivot_table['% del total'] = (pivot_table['count'] / total_productos) * 100

# Agregar una fila para el total de productos reciclados
total_reciclado = pivot_table['count'].sum()
total_row = pd.DataFrame({'reciclado': ['Total productos reciclados'], 'count': [total_reciclado], '% del total': [total_reciclado / total_productos * 100]})

# Concatenar la fila total al DataFrame pivot_table
pivot_table = pd.concat([pivot_table, total_row], ignore_index=True)

pivot_table

,reciclado,count,% del total
0,algodon reciclado,50,6.944444
1,poliester reciclado,40,5.555556
2,poliamida reciclada,35,4.861111
3,Total productos reciclados,125,17.361111


In [125]:
#pivot_table.to_csv('reciclado_camisetas2.csv')

In [126]:
tshirt['reciclado'] = tshirt['reciclado'].str.replace('reciclado', '').str.replace('reciclada', '')

In [127]:
columns_to_keep = ["comprobacion2", "algodon de cultivo organico certificado OCS","poliester reciclado","algodon reciclado","poliamida reciclada certificada RCS", "acrilico reciclado", "elastano reciclado", "certificacion", "porcentaje", "reciclado"]

filtered_df = tshirt[columns_to_keep]
#filtered_df.head(20)

In [128]:
columns_to_keep = ["index","product_name", "price", "made_in", "standard_soc_ma", "material","num_materials","num_material","material_secundario","num_material_secundario","comprobacion2","certificacion","porcentaje", "porcentaje"]

final_df = tshirt[columns_to_keep]
#final_df.tail(20)

In [129]:
#tshirt['certificacion'] = tshirt.apply(asignar_certificacion, axis=1)


In [130]:
#tshirt["certificacion"].value_counts()

In [131]:
#tshirt.info()

In [132]:
tshirt["certificacion"].value_counts()

certificacion
None    526
OCS     159
RCS      35
Name: count, dtype: int64

## FINAL TSHIRT DF: 

In [133]:
tshirt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 720 entries, 0 to 728
Data columns (total 38 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   index                                        720 non-null    int64  
 1   product_name                                 720 non-null    object 
 2   price                                        720 non-null    object 
 3   desciption                                   720 non-null    object 
 4   made_in                                      720 non-null    object 
 5   standard_soc_ma                              720 non-null    bool   
 6   comprobacion                                 720 non-null    object 
 7   algodon                                      622 non-null    object 
 8   poliamida                                    48 non-null     object 
 9   elastano                                     218 non-null    object 
 10  lino   

In [134]:
tshirt = tshirt.drop(columns=['desciption', "comprobacion", "comprobacion2"])

In [135]:
tshirt.rename(columns={
    "material": "main_txt",
    "num_materials": "total_composition",
    "num_material": "1_composition",
    "material_secundario": "second_txt",
    "num_material_secundario": "2_composition",
    "reciclado": "txt_recycled",
    "algodon de cultivo organico certificado OCS": "cotton_OCS",
    "poliester reciclado": "rec_poliester",
    "algodon reciclado": "rec_cotton",
    "poliamida reciclada certificada RCS": "rec_poliamida",
    "acrilico reciclado": "rec_acrilico",
    "elastano reciclado" : "rec_elastano"
}, inplace=True)

In [136]:
#tshirt.info()

In [137]:
# Función para quitar el símbolo '%'
def remove_percent(x):
    if isinstance(x, str):
        return x.replace('%', '')
    return x

In [138]:
tshirt = tshirt.applymap(remove_percent)

In [139]:
#tshirt.info()

In [140]:
columns_to_convert = [
    'algodon', 'poliamida', 'elastano', 'lino', 'viscosa', 
    'poliester', 'modal', 'otras', 'liocel', 'fibra', 
    'acrilico', 'lana', 'seda', 
    'cotton_OCS', 'rec_poliester', 'rec_cotton', 'rec_poliamida', 
    'rec_acrilico', 'rec_elastano', 'porcentaje'
]

# Convertir las columnas especificadas a numérico
for column in columns_to_convert:
    tshirt[column] = pd.to_numeric(tshirt[column], errors='coerce')

In [141]:
tshirt['price'] = pd.to_numeric(tshirt['price'].str.replace(',', '.'), errors='coerce')

In [142]:
#tshirt.info()

In [143]:
columns1 = ['algodon', 'poliamida', 'elastano', 'lino', 'viscosa', 'poliester', 'modal', 'otras', 'liocel',
                     'fibra', 'acrilico', 'lana', 'seda', '1_composition', '2_composition', 'cotton_OCS', 'rec_poliester',
                     'rec_cotton', 'rec_poliamida', 'rec_acrilico','rec_elastano', 'porcentaje']

tshirt[columns1] = tshirt[columns1].apply(lambda x: x / 100)

In [144]:
#pd.set_option('display.max_columns', None)
#tshirt.head()

In [145]:
# Calcular el total de certificaciones OCS y el total de algodón en todas las prendas
total_OCS = tshirt['OCS'].eq('OCS').sum()
total_algodon = tshirt['algodon'].sum()

# Calcular el porcentaje de OCS en función del total de algodón
porcentaje_OCS_total = (total_OCS / total_algodon) * 100
porcentaje_OCS_total

17.563995548135782

In [146]:
tshirt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 720 entries, 0 to 728
Data columns (total 35 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              720 non-null    int64  
 1   product_name       720 non-null    object 
 2   price              720 non-null    float64
 3   made_in            720 non-null    object 
 4   standard_soc_ma    720 non-null    bool   
 5   algodon            622 non-null    float64
 6   poliamida          48 non-null     float64
 7   elastano           218 non-null    float64
 8   lino               17 non-null     float64
 9   viscosa            34 non-null     float64
 10  poliester          112 non-null    float64
 11  modal              14 non-null     float64
 12  otras              4 non-null      float64
 13  liocel             9 non-null      float64
 14  fibra              3 non-null      float64
 15  acrilico           2 non-null      float64
 16  lana               3 non-null  

In [147]:
#tshirt.to_csv("tshirt_final.csv", index=True)

### EDA

In [148]:
total_rows = len(tshirt)
ocs_percentage = (tshirt['OCS'].value_counts() / total_rows) * 100
rcs_percentage = (tshirt['RCS'].value_counts() / total_rows) * 100

print(f" el % de OCS es: {ocs_percentage} y el de RCS es {rcs_percentage}")

 el % de OCS es: OCS
       85.972222
OCS    14.027778
Name: count, dtype: float64 y el de RCS es RCS
       87.083333
RCS    12.916667
Name: count, dtype: float64
